In [1]:
import pandas as pd
import os
import glob
import numpy as np
import statistics as stats
import scipy
import mpra_tools.predicted_occupancy as po
import mpra_tools.fasta_utils as fu
import math
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import random
from sklearn.metrics import precision_recall_curve, roc_curve, f1_score, precision_score, recall_score

In [2]:
activity_df = pd.read_csv("Data/activity.csv", index_col=0)
retinopathy_df = pd.read_csv("Data/retinopathy_reformatted.txt", sep='\t', index_col=1)
test_labels = activity_df[activity_df['test_set'] | activity_df['cnn_validation_set']]['label']
train_labels = activity_df[~activity_df['label'].isin(test_labels)]['label']
L = 164
print(len(activity_df), "samples")

118364 samples


In [14]:
retinopathy_df.head()

,Unnamed: 0,library,genotype,activity_mean,activity_std,n_observations,activity_mu,activity_sigma,pvalue,qvalue,activity_class,variant_type,chip_peak_id,sequence,activity_vs_basal,expression_log2,activity_bin
label,,,,,,,,,,,,,,,,,
CRX.1003,0,rho,WT,0.212773,0.069493,12.0,-1.598208,0.318364,4.222015e-02,5.154038e-02,inactive,wildtype,CRX.1003,CCAGCGTTCCTCCCATCTGTAGCAGTCAGGGTTCTTAAGTAGCAAA...,0.771869,-0.373573,Inactive
CRX.1004,2,rho,WT,1.056873,0.300882,12.0,0.016349,0.279162,8.650884e-13,2.948816e-12,weak_enhancer,wildtype,CRX.1004,CTAACTTCCGTATCCACTGCCCCCACCCCCACCCCTGTGTCCTCAC...,3.833980,1.938843,WeakEnhancer
CRX.1007,4,rho,WT,0.010203,0.010766,12.0,-4.959274,0.865048,7.189995e-09,1.738871e-08,strong_silencer,wildtype,CRX.1007,CCTGCTAACCTGACCTTCTTATTCATAGATGAGGAAATAGAGGCTT...,0.037011,-4.755889,Silencer
CRX.101,6,rho,WT,0.910381,0.369439,12.0,-0.170115,0.390443,3.512835e-08,7.924102e-08,weak_enhancer,wildtype,CRX.101,CACCGCCCACAGGACACCAGCCCTTGGGTTTTATAATCTCTCACAG...,3.302558,1.723584,WeakEnhancer
CRX.1011,7,rho,WT,0.663029,0.559840,12.0,-0.680046,0.733635,8.350663e-03,1.092875e-02,weak_enhancer,wildtype,CRX.1011,GGCAGGCTTTGTTGTCGTCCGAGCAGCTGTGACCGTTGGGGCTGAG...,2.405247,1.266185,WeakEnhancer


In [15]:
fimo_df = pd.concat([pd.read_csv('Data/fimo_eLife_activity/fimo.tsv', sep='\t'),pd.read_csv('Data/fimo_eLife_retinopathy/fimo.tsv', sep='\t')], ignore_index=True)
del fimo_df['motif_alt_id']
fimo_df.dropna(inplace=True)
fimo_df['motif_id'] = fimo_df['motif_id'].map(lambda x: x.split('_')[0])
fimo_df = fimo_df.astype({'start':int, 'stop':int})
print(len(fimo_df),"motifs")
list(set(fimo_df['motif_id']))

273886 motifs


['NDF1', 'CRX', 'RORB', 'GFI1', 'MAZ', 'NRL', 'MEF2D', 'RAX']

### Transform Each Sequence into a sentence

In [16]:
# main dataset
labels = fimo_df.groupby(by="sequence_name")
sentences = dict()
IVAL = 5.0

for name, df in labels:
    sdf = df.sort_values(by="start")
    i = 1
    s = []
    for index, row in sdf.iterrows():
        d = row['start']-i
        if d > 0:
            xIVALmer = math.ceil(d/IVAL)
            s.append(xIVALmer)
        s.append(row['motif_id']+row['strand'])
        i = row['stop']+1
    if i < L:
        xIVALmer = math.ceil((L-i)/IVAL)
        s.append(xIVALmer)
    sentences[name] = s


### Get word counts for each document class

In [5]:
bins = ['Silencer','Inactive','WeakEnhancer','StrongEnhancer']

In [6]:
#Total word counts for each class
counts = dict(
    [(b,Counter()) for b in bins]
)


#Proportion of word counts N(w)/N(V)
p_wc = dict(
    [(b,defaultdict(float)) for b in bins]
)


#Counts all words for each class
train_labels = train_labels[train_labels.isin(sentences.keys())]
train_bins = activity_df.loc[train_labels.index]['activity_bin']

for i in train_labels.index:
    s = sentences[train_labels.loc[i]]
    counts[train_bins.loc[i]].update(s)


#Convert raw counts to naive probabilities
for b in bins:
    p_wc[b] = {k: v / counts[b].total() for k,v in counts[b].items()}
    
    
#Get class probabilities
bin_counts = activity_df.loc[train_labels.index]['activity_bin'].value_counts()
p_c = dict(
    [(b,bin_counts[b]/sum(bin_counts)) for b in bins]
)


#Get length of alphabet
alphabet = list(set(np.concatenate(list(sentences.values()))))
V = len(alphabet)

### Get class probabilities for all docs in the testing set

In [48]:
preds_test = dict()
truths_test = dict()

test_labels = test_labels[test_labels.isin(sentences.keys())]
test_bins = activity_df.loc[test_labels.index]['activity_bin']

for i in test_labels.index:
    #Inititial probabilities for each doc P(c_i)
    probs = dict([(b,p_c[b]) for b in bins])
    for b in bins:
        for w in sentences[test_labels.loc[i]]:
            # Probability that a word appears in the doc.  Log transform means we can add
            probs[b] = probs[b] * (counts[b][w]+1)/(counts[b].total()+V)
    norm = sum(probs.values())
    preds_test[test_labels.loc[i]] = np.array(list(probs.values())) / norm
    truths_test[test_labels.loc[i]] = bins.index(test_bins.loc[i])





In [49]:
preds_ret = dict()
truths_ret = dict()

ret_labels = retinopathy_df[retinopathy_df.index.isin(sentences.keys())].index
ret_bins = retinopathy_df.loc[ret_labels]['activity_bin']

for i in ret_labels:
    #Inititial probabilities for each doc P(c_i)
    probs = dict([(b,p_c[b]) for b in bins])
    for b in bins:
        for w in sentences[i]:
            # Probability that a word appears in the doc.  Log transform means we can add
            probs[b] = probs[b] * (counts[b][w]+1)/(counts[b].total()+V)
    norm = sum(probs.values())
    preds_ret[i] = np.array(list(probs.values())) / norm
    truths_ret[i] = bins.index(ret_bins.loc[i])




In [50]:
averages = ['micro', 'macro','weighted']

print("retinopathy test")
t = list(truths_ret.values())
p = [a.argmax() for a in preds_ret.values()]

correct = 0
for truth, pred in zip(t,p):
    if truth == pred:
        correct += 1
        
correct = correct/len(t)
print("correct:", round(correct,2))


f1_ret = [f1_score(t,p, labels=range(len(bins)), average=a) for a in averages] + [correct]

retinopathy test
correct: 0.32


In [51]:

t = list(truths_test.values())
p = [a.argmax() for a in preds_test.values()]

correct = 0
for truth, pred in zip(t,p):
    if truth == pred:
        correct += 1
        
correct = correct/len(t)

f1_test = [f1_score(t,p, labels=range(len(bins)), average=a) for a in averages] + [correct]


In [52]:
bayes_preformance = pd.DataFrame(
    data = [f1_ret,f1_test],
    columns=averages+['correct'],
    index=['Retinopathy Set', 'Test Set']
)

In [40]:
Clf_performance = pd.read_csv("Data/cnn_dataset_performance_metrics.txt", sep ='\t', index_col=[0,1])
clf_test = Clf_performance[Clf_performance['test_set']=='test_set'].copy()
clf_ret = Clf_performance[Clf_performance['test_set']=='test_retinopathy'].copy()
clf_test.drop(columns=['test_set','nseqs_train','nseqs_test'],inplace=True)
clf_ret.drop(columns=['test_set','nseqs_train','nseqs_test'],inplace=True)


In [41]:
clf_test.groupby(level=0).agg('median')

,micro,macro,weighted,Si,In,WE,SE
dataset,,,,,,,
GenomicOnly,0.350914,0.145846,0.211773,0.000000,0.000000,0.506837,0.071107
Round1,0.362166,0.172657,0.241627,0.000000,0.000000,0.504582,0.143642
Round2,0.354430,0.135631,0.191908,0.000000,0.015566,0.525931,0.000000
Round3a,0.371308,0.268965,0.370702,0.000000,0.271604,0.327779,0.467166
Round3aNoRound2,0.401547,0.282342,0.395492,0.000000,0.231200,0.439486,0.456208
Round3b,0.440928,0.341651,0.446962,0.067816,0.355312,0.387801,0.563356
Round3c,0.341772,0.175502,0.215086,0.000000,0.171492,0.476005,0.058765
Round4a,0.317159,0.174829,0.223549,0.066964,0.039871,0.518804,0.065261
Round4b,0.345992,0.282339,0.322559,0.144871,0.266884,0.477378,0.234446


In [42]:
clf_ret.groupby(level=0).agg('median')

,micro,macro,weighted,Si,In,WE,SE
dataset,,,,,,,
GenomicOnly,0.483749,0.176835,0.335919,0.000000,0.004785,0.657935,0.033794
Round1,0.470981,0.193926,0.340705,0.016178,0.000000,0.648187,0.109796
Round2,0.473012,0.184800,0.333342,0.000000,0.093793,0.645405,0.000000
Round3a,0.183401,0.160619,0.155211,0.002062,0.206688,0.215517,0.214100
Round3aNoRound2,0.281776,0.219404,0.271737,0.000000,0.215467,0.450504,0.222348
Round3b,0.232443,0.207965,0.219153,0.031886,0.220859,0.308574,0.254433
Round3c,0.432966,0.192154,0.328411,0.000000,0.102592,0.622209,0.049745
Round4a,0.505514,0.314424,0.473670,0.522945,0.084478,0.626301,0.010363
Round4b,0.370284,0.338656,0.396018,0.495700,0.178315,0.421774,0.246826


In [53]:
bayes_preformance

,micro,macro,weighted,correct
Retinopathy Set,0.318545,0.218040,0.258243,0.318545
Test Set,0.262172,0.219618,0.275600,0.262172


In [ ]:
seqs = activity_df['sequence']

motifs = 0

for seq in seqs[:1000]:
    motif_dict = po.get_occupied_sites_and_tfs(po.total_landscape(seq,ewm,9), cutoff=0.2)
    motifs += len(motif_dict)

In [9]:
correct = 0
for truth in t:
    pred = random.choices([0,1,2,3],weights = list(p_c.values()))[0]
    if truth == pred:
        correct += 1

correct/len(t)

0.21273408239700375

In [10]:
ewm = po.read_pwm_to_ewm("Data/Motifs/eLifeMotifs.meme")

In [11]:
seqs = activity_df['sequence']

motifs = 0

for seq in seqs[:1000]:
    motif_dict = po.get_occupied_sites_and_tfs(po.total_landscape(seq,ewm,9), cutoff=0.2)
    motifs += len(motif_dict)

In [12]:
motifs

3069